## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-03-20-59-27 +0000,nypost,Mamdani pushes for ‘Department of Community Sa...,https://nypost.com/2026/02/03/us-news/mamdani-...
1,2026-02-03-20-58-02 +0000,nypost,Wild casino brawl sees gamblers grapple with w...,https://nypost.com/2026/02/03/us-news/wild-har...
2,2026-02-03-20-57-18 +0000,nyt,U.K. Investigates Peter Mandelson Over Jeffrey...,https://www.nytimes.com/2026/02/03/world/europ...
3,2026-02-03-20-54-28 +0000,nypost,Inside the heart-sinking moment Savannah Guthr...,https://nypost.com/2026/02/03/us-news/inside-t...
4,2026-02-03-20-53-42 +0000,nyt,"‘Don Colossus,’ a Golden Statue of President T...",https://www.nytimes.com/2026/02/03/technology/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2409/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
46,trump,42
27,epstein,26
153,house,15
552,new,14
246,will,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
190,2026-02-03-14-44-00 +0000,wsj,Bill and Hillary Clinton made a new offer to g...,https://www.wsj.com/politics/policy/bill-hilla...,125
344,2026-02-02-22-44-00 +0000,wsj,Trump demanded Monday that House lawmakers bac...,https://www.wsj.com/politics/policy/trump-push...,111
27,2026-02-03-20-18-00 +0000,wsj,Bill and Hillary Clinton reached a deal to app...,https://www.wsj.com/politics/clintons-reach-de...,104
42,2026-02-03-20-04-04 +0000,startribune,House passes bill to end the partial governmen...,https://www.startribune.com/gop-leaders-labor-...,96
88,2026-02-03-18-44-52 +0000,nypost,Putin breaks Trump’s cease-fire after just 4 d...,https://nypost.com/2026/02/03/world-news/putin...,86


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
190,125,2026-02-03-14-44-00 +0000,wsj,Bill and Hillary Clinton made a new offer to g...,https://www.wsj.com/politics/policy/bill-hilla...
344,96,2026-02-02-22-44-00 +0000,wsj,Trump demanded Monday that House lawmakers bac...,https://www.wsj.com/politics/policy/trump-push...
83,67,2026-02-03-19-02-45 +0000,nyt,"Authorities Believe Savannah Guthrie’s Mother,...",https://www.nytimes.com/2026/02/03/us/savannah...
43,63,2026-02-03-20-03-43 +0000,cbc,Disney parks chief Josh D'Amaro replaces Bob I...,https://www.cbc.ca/news/entertainment/disney-c...
152,57,2026-02-03-16-19-18 +0000,bbc,Son of Norway's crown princess hears details o...,https://www.bbc.com/news/articles/ckgn8dnxjgwo...
58,47,2026-02-03-19-41-36 +0000,nypost,Jeffrey Epstein asked woman to buy Snow White ...,https://nypost.com/2026/02/03/business/jeffrey...
11,41,2026-02-03-20-44-40 +0000,nypost,Chopped-up body parts found in bag belonged to...,https://nypost.com/2026/02/03/us-news/chopped-...
332,39,2026-02-02-23-08-00 +0000,wsj,The Trump administration is preparing to devel...,https://www.wsj.com/politics/policy/trump-admi...
135,37,2026-02-03-17-16-51 +0000,nypost,Credit Suisse investigator claims he was fired...,https://nypost.com/2026/02/03/us-news/credit-s...
35,35,2026-02-03-20-11-53 +0000,nyt,A Community Space for Team USA at the Winter O...,https://www.nytimes.com/2026/02/03/sports/olym...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
